# Tafeln

In [1]:
%matplotlib inline
from IPython.display import display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

con = sqlite3.connect("../data/CongoDB.sqlite")

/home/dseidensticker/anaconda3/lib/python3.6/site-packages/matplotlib/font_manager.py:278: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [3]:
pl = pd.read_csv('../plates/plates.csv')
display(pl.head(1))

,TAF,OBJ,SITE,FEAT,NOTE,IND
0,1,1,BRU,85/101,NaN,1


In [4]:
sql = """SELECT
           t_Obj.objID,
           t_Ort.ort_kurz AS SITE,
           t_ort.ort_name AS Fundort,
           't_ort'.'Kat-Nr' AS 'Fpl.',
           't_Ort'.'ort_kurz' || ' ' || 't_Komplex'.'bef_nr' AS Befund,
           t_Komplex.bef_nr AS FEAT,
           t_Komplex.bef_DissDS_KatNr AS 'Kat.-Nr.',
           t_Obj.Individuum AS IND,
           t_Obj.Typ AS STYLE
       FROM (t_Ort INNER JOIN t_Komplex ON t_Ort.ortID = t_Komplex.ortID)
           INNER JOIN t_Obj ON t_Komplex.komplexID = t_Obj.komplexID
       WHERE (((t_Ort.ort_lit)='DS'))""".replace('\n',' ')
df = pd.read_sql(sql, con)
display(df.head(1))

,objID,SITE,Fundort,Fpl.,Befund,FEAT,Kat.-Nr.,IND,STYLE
0,10,BAT,Batanga,209,BAT 85/101,85/101,NaN,3,MBN


In [5]:
df[df['SITE'] == 'LKK']

,objID,SITE,Fundort,Fpl.,Befund,FEAT,Kat.-Nr.,IND,STYLE
169,204,LKK,Lokekya,188,LKK 85/101,85/101,NaN,None,BKW (?)


> jetzt ein LeftJoin auf pl ==>> hier passt alles:

In [6]:
df_merge_inner = pd.merge(pl, df, on=['SITE', 'FEAT', 'IND'], how='inner')
display(df_merge_inner.head())
df_merge_inner.to_csv('../data/processed/10_TafNr-objID_Liste.csv', index=False)

,TAF,OBJ,SITE,FEAT,NOTE,IND,objID,Fundort,Fpl.,Befund,Kat.-Nr.,STYLE
0,1,1,BRU,85/101,NaN,1,68,Bruxelles-Nganda,186,BRU 85/101,NaN,BKW
1,1,2,LKK,85/101,NaN,NaN,204,Lokekya,188,LKK 85/101,NaN,BKW (?)
2,1,3,BOB,85/101,NaN,a,5727,Bobangi,189,BOB 85/101,NaN,None
3,1,4,BOB,85/101,NaN,b,62,Bobangi,189,BOB 85/101,NaN,BOT (?)
4,2,1,BKW,85/101,NaN,NaN,1469,Bokwango,190,BKW 85/101,NaN,BKW


### Nicht automatisch abgeglichen:

In [7]:
df_merge_outer = pd.merge(pl, df, on=['SITE', 'FEAT', 'IND'], how='left')
# display(df_merge_outer.head())

In [8]:
a = df_merge_outer[df_merge_outer['objID'].isnull()]
len(a)

1

In [9]:
display(a.head(10))

,TAF,OBJ,SITE,FEAT,NOTE,IND,objID,Fundort,Fpl.,Befund,Kat.-Nr.,STYLE
504,41,9,BOG,87/103,NaN,"5,2",NaN,NaN,NaN,NaN,NaN,NaN


## erzeugte Doubletten:

In [10]:
# df_merge_inner[df_merge_inner['objID'].duplicated(keep = False)]
# zeigen nur noch ausgezählte Stücke an

## Auflistung für Tafel-Untertitel

In [11]:
# damit alle Zeilen angezeigt werden:
# pd.set_option('display.max_rows', None)

pl.pivot_table(values = 'OBJ', index = ['TAF', 'SITE', 'FEAT'], aggfunc = [min, max]).head()
#.reset_index()

min max
                OBJ OBJ
TAF SITE FEAT          
1   BOB  85/101   3   4
    BRU  85/101   1   1
    LKK  85/101   2   2
2   BKW  85/101   1   2
    ZAM  85/101   3   6

> view in der DB erzeugen:

```
CREATE VIEW v_obj
AS
SELECT
 t_Obj.objID,
 t_Ort.ort_kurz AS SITE,
 t_Komplex.bef_nr AS FEAT,
 t_Obj.Individuum AS IND,
 t_Obj.GefEinheit AS GE
FROM (t_Ort INNER JOIN t_Komplex ON t_Ort.ortID = t_Komplex.ortID)
 INNER JOIN t_Obj ON t_Komplex.komplexID = t_Obj.komplexID
```

## Auflistung für Katalog (siehe Wotzka 1995):

In [11]:
stilgr = pd.read_csv("../lit/Wotzka1995_StilGrKuerzel.csv", encoding='Latin1')
stilgr.head(2)

,Typ,Stilgruppe,Wotzka1995,Seidensticker,Lit
0,BAN,Bangui,NaN,3.2.1.11,NaN
1,BTM,Batalimo-Maluba,NaN,3.2.1.1,NaN


In [12]:
df_merge_inner.head(2)

,TAF,OBJ,SITE,FEAT,NOTE,IND,objID,Fundort,Fpl.,Befund,Kat.-Nr.,STYLE
0,1,1,BRU,85/101,NaN,1,68,Bruxelles-Nganda,186,BRU 85/101,NaN,BKW
1,1,2,LKK,85/101,NaN,NaN,204,Lokekya,188,LKK 85/101,NaN,BKW (?)


In [13]:
plList = df_merge_inner[['TAF', 'OBJ', 'Fundort', 'Fpl.', 'Befund', 'Kat.-Nr.', 'STYLE']]
plList.head()

,TAF,OBJ,Fundort,Fpl.,Befund,Kat.-Nr.,STYLE
0,1,1,Bruxelles-Nganda,186,BRU 85/101,NaN,BKW
1,1,2,Lokekya,188,LKK 85/101,NaN,BKW (?)
2,1,3,Bobangi,189,BOB 85/101,NaN,None
3,1,4,Bobangi,189,BOB 85/101,NaN,BOT (?)
4,2,1,Bokwango,190,BKW 85/101,NaN,BKW


In [14]:
# ohne das hier kommt eine etwas nervige Warnung, die mir aber egals ist
pd.options.mode.chained_assignment = None  # default='warn'

di = dict(zip(stilgr['Typ'],stilgr['Stilgruppe']))

plList["STYLE"] = plList["STYLE"].replace(di, regex = True)
plList["STYLE"] = plList["STYLE"].replace(np.nan, "-")
plList["Kat.-Nr."] = plList["Kat.-Nr."].replace(np.nan, "-")

plList = plList.rename(columns={'Befund': 'Komplex', 
                                'STYLE': 'Stilgruppe',
                                'TAF': 'Taf.',
                                'OBJ': 'Nr.'})

plList.to_latex('../output/tabs/10_TafObj_Liste.tex', longtable = True, index = False, escape = False)

plList.head(20)

,Taf.,Nr.,Fundort,Fpl.,Komplex,Kat.-Nr.,Stilgruppe
0,1,1,Bruxelles-Nganda,186,BRU 85/101,-,Bokwango
1,1,2,Lokekya,188,LKK 85/101,-,Bokwango (?)
2,1,3,Bobangi,189,BOB 85/101,-,-
3,1,4,Bobangi,189,BOB 85/101,-,Botendo (?)
4,2,1,Bokwango,190,BKW 85/101,-,Bokwango
5,2,2,Bokwango,190,BKW 85/101,-,Botendo
6,2,3,Zamba,191,ZAM 85/101,-,-
7,2,4,Zamba,191,ZAM 85/101,-,-
8,2,5,Zamba,191,ZAM 85/101,-,-
9,2,6,Zamba,191,ZAM 85/101,-,-


### Reihenfolge entsprechend Stilgruppenliste

In [15]:
df_merge_inner.pivot_table(values = 'IND', 
                           index = ['STYLE', 'Fundort', 'TAF', 'OBJ'], 
                           aggfunc = len).reset_index().head()

,STYLE,Fundort,TAF,OBJ,IND
0,BAN,Balongoi,20,7,1
1,BAN,Balongoi,20,8,1
2,BAN,Bangui,21,1,1
3,BAN,Libenge,16,1,1
4,BAN (?),Kouango,24,2,1
